## Importing required libraries

In [1]:
import pandas as pd

In [6]:
import numpy as np
import matplotlib.pyplot as plt

Importing the preprocessed and saved data

In [4]:
data = pd.read_csv(r"C:\Users\Lenovo\Documents\jupyter notebook DATA SCIENCE\ML project\Project on HR Analytics data\eda_data")

In [5]:
data.head()

,Unnamed: 0,department,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,0,8,2,0,1,1,35,5.0,8,1,0,49,0
1,1,7,1,1,2,1,30,5.0,4,0,0,60,0
2,2,8,1,1,1,1,34,3.0,7,0,0,50,0
3,3,8,1,1,2,2,39,1.0,10,0,0,50,0
4,4,6,1,1,2,1,45,3.0,2,0,0,73,0


In [7]:
# Removing the 'Unnamed column' 
data = data.drop('Unnamed: 0',axis=1)

In [8]:
data.head()

,department,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,8,2,0,1,1,35,5.0,8,1,0,49,0
1,7,1,1,2,1,30,5.0,4,0,0,60,0
2,8,1,1,1,1,34,3.0,7,0,0,50,0
3,8,1,1,2,2,39,1.0,10,0,0,50,0
4,6,1,1,2,1,45,3.0,2,0,0,73,0


In [9]:
# verifying if there is any null values
data.isna().sum()

department              0
education               0
gender                  0
recruitment_channel     0
no_of_trainings         0
age                     0
previous_year_rating    0
length_of_service       0
KPIs_met >80%           0
awards_won?             0
avg_training_score      0
is_promoted             0
dtype: int64

Splitting X and Y

In [10]:
X = data.drop('is_promoted',axis=1)
y = data['is_promoted']

Splitting data to train and test data

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.33,random_state=42)

## SMOTE

In [12]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X_res,y_res = sm.fit_resample(X_train,y_train)

## XGBOOST 

In [13]:
import xgboost as xgb
classifier = xgb.XGBClassifier()
classifier.fit(X_res, y_res)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [14]:
y_pred = classifier.predict(X_test)

Getting accuracy,Precision and F1 score

In [15]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred,y_test)

0.8693536794382706

In [16]:
from sklearn.metrics import f1_score
f1_score(y_pred,y_test)

0.40939765058735317

In [19]:
from sklearn.metrics import precision_score
precision_score(y_pred,y_test)

0.5556309362279511

In [20]:
#precision,Recall,F1 Score
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_test,y_pred)

(array([0.95790488, 0.32409972]),
 array([0.89718895, 0.55563094]),
 array([0.92655332, 0.40939765]),
 array([16613,  1474], dtype=int64))

In [21]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.96      0.90      0.93     16613
           1       0.32      0.56      0.41      1474

    accuracy                           0.87     18087
   macro avg       0.64      0.73      0.67     18087
weighted avg       0.91      0.87      0.88     18087



Precision , Recall and F1 score is better in XGBoost Model than Random Forest forest

XGBoost Data Matrix

Now converting the dataset into an optimized data structure called Dmatrix that XGBoost supports and gives it acclaimed performance and efficiency gains


In [22]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [23]:
params = {"objective":"binary:logistic",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}
cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=5000,early_stopping_rounds=5,metrics="error", as_pandas=True, seed=123)

In [24]:
cv_results

,train-error-mean,train-error-std,test-error-mean,test-error-std
0,0.085170,0.000951,0.08517,0.001901
1,0.081356,0.004934,0.08101,0.007109


In [ ]:
#END